In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
from matplotlib import pyplot as plt
%config InlineBackend.figure_format = 'retina'

In [ ]:
import pandas as pd
data = pd.read_csv('Soil_moisture_project.csv')
# print(data.info())


y1 = data['Soil Moisture 10cm,%']
y2 = data['Soil Moisture 20cm,%']

data.drop(['Date', 'Plot_Name'], axis=1, inplace=True)
# print(data.head())

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 

checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 

#splitting dataset before applying NN
X_train, X_holdout, y_train, y_holdout = train_test_split(data.values, y1, test_size=0.3,random_state=17)
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(50, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(100, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(100, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(100, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

NN_model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)


In [ ]:
predictions_test = NN_model.predict(X_holdout)
# predictions_test = NN_model.predict(X_train)


plt.scatter(predictions_test,y_holdout)


In [ ]:
from scipy import stats
x=y_holdout
y=np.reshape(predictions_test,(32,))
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
line = slope * x+intercept
def plot_scatter(axes,x,y,color,xlabel,ylabel,title):
    axes.scatter(x,y,color=color,s=9)
    axes.set_xlabel(xlabel)
    axes.set_ylabel(ylabel)
    axes.tick_params('x',direction='in')
    axes.tick_params('y',direction='in')
    axes.set_title(title)

fig,ax1=plt.subplots(figsize=(9,6))
plot_scatter(ax1,y_holdout,predictions_test,'b','Actual[%]','Prediction[%]','Soil Moisture at 10cm')
ax1.plot(x, line, 'orange',linestyle='None', label='y={:.2f}x+({:.2f})\nR2={:.4f}'.format(slope,intercept,r_value))
ax1.set_xlim(10,28)
ax1.set_ylim(10,28)
ax1.plot( [10,28],[10,28],color='r')
ax1.tick_params(direction='in')
plt.legend()

# plt.savefig('soil_moisture_10cm.png',dpi=300)


For 20 cm depth

In [ ]:
#splitting dataset before applying NN
X_train, X_holdout, y_train, y_holdout = train_test_split(data.values, y2, test_size=0.3,random_state=17)
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

NN_model.fit(X_train, y_train, epochs=700, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)


In [ ]:
from scipy import stats
predictions_test = NN_model.predict(X_holdout)
x=y_holdout
print(y_holdout.shape)
y=np.reshape(predictions_test,(32,))
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
line = slope * x+intercept
def plot_scatter(axes,x,y,color,xlabel,ylabel,title):
    axes.scatter(x,y,color=color,s=9)
    axes.set_xlabel(xlabel)
    axes.set_ylabel(ylabel)
    axes.tick_params('x',direction='in')
    axes.tick_params('y',direction='in')
    axes.set_title(title)

fig,ax1=plt.subplots(figsize=(9,6))
plot_scatter(ax1,y_holdout,predictions_test,'b','Actual[%]','Prediction[%]','Soil Moisture at 20cm')
ax1.plot(x, line, 'g',linestyle='None', label='y={:.2f}x+({:.2f})\nR2={:.4f}'.format(slope,intercept,r_value))
ax1.set_xlim(18,32)
ax1.set_ylim(18,32)
ax1.plot( [18,32],[18,32],color='r')
ax1.tick_params(direction='in')
plt.legend()

# plt.savefig('soil_moisture_20cm.png',dpi=300)